Imports

In [40]:
from models import ff_eucl, ff_hyp
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import geoopt
from time import time
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
import helper
torch.cuda.is_available()

False

CUDA check

In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using {}'.format(device))

Using cpu


Data Transformation

In [42]:
transform = transforms.Compose([transforms.ToTensor(),
                                #transforms.Normalize((0.1307,), (0.3081,)), 
                              ])

Training and Test data from MNIST data set

In [43]:
train_set = datasets.MNIST('PATH_TO_STORE_TRAINSET', download=True, train=True, transform=transform)
test_set = datasets.MNIST('PATH_TO_STORE_TESTSET', download=True, train=False, transform=transform)

size = len(train_set)
print(size)

train_data, val_data = torch.utils.data.random_split(train_set, [int(size-size*0.2), int(size*0.2)])

trainloader = torch.utils.data.DataLoader(train_data, batch_size=512, shuffle=True)
valloader = torch.utils.data.DataLoader(val_data, batch_size=512, shuffle= True)
testloader = torch.utils.data.DataLoader(test_set, batch_size=512, shuffle= True)

60000


Initializing the model

In [56]:
model = ff_eucl.EuclFF(784, 512, 256, 10, nn.ReLU())
ball = geoopt.PoincareBall()
images, labels = next(iter(trainloader))
images = images.view(images.shape[0], -1)
grid = torchvision.utils.make_grid(images)
tb = SummaryWriter()
tb.add_image("images", grid)
tb.add_graph(model, images)
model.to(device)

EuclFF(
  (act_fn): ReLU()
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)

Hyperparameters

In [45]:

learning_rate = 4e-1 #learning rate for ReLU activation function
#learning_rate = 2e-1 #current learning rate for model without activation functions
momentum = 0.9
weight_decay = 5e-4

Loss Function

In [46]:
criterion = nn.CrossEntropyLoss()
# #criterion = nn.NLLLoss()
# images, labels = next(iter(trainloader))
# images, labels = images.to(device), labels.to(device)
# images = 0.0357*images.view(images.shape[0], -1)
# print(images)

# out = model(images) #output
# print(out)
# loss = criterion(out, labels) #calculate the loss

Optimizer

In [47]:
#optimizer = geoopt.optim.RiemannianSGD(model.parameters(), lr=learning_rate, momentum=momentum)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

In [48]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [49]:

# time0 = time()
epochs = 10
for epoch in range(epochs):
    model.train()
    train_loss = 0
    total_correct = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        # Flatten MNIST images into a 784 long vector
        images = images.view(images.shape[0], -1)
        #images = ball.projx(images.view(images.shape[0], -1))
        # Training pass
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)  
        train_loss += loss.item()
        total_correct += get_num_correct(output, labels)
        #backpropagation
        loss.backward()      
        #Weight optimization
        optimizer.step()  

    val_loss = 0
    val_correct = 0
    model.eval()
    for  images, labels in valloader:
        images, labels = images.to(device), labels.to(device)
        # Flatten MNIST images into a 784 long vector
        images = images.view(images.shape[0], -1)
        #images = ball.projx(images.view(images.shape[0], -1))
        output = model(images)
        loss = criterion(output, labels)  
        val_loss += loss.item()
        val_correct += get_num_correct(output, labels)
        
    tb.add_scalar("Training Loss", train_loss, epoch)
    tb.add_scalar("Validation Loss", val_loss, epoch)
    tb.add_scalar("Training Accuracy", total_correct/len(train_data), epoch)
    tb.add_scalar("Validation Accuracy", val_correct/len(val_data), epoch)

    print("epoch:", epoch, "training loss:",train_loss, "validation loss:", val_loss,
    "training accuracy:", total_correct/len(train_data), "validation accuracy:", val_correct/len(val_data))
    
tb.close()
#     else:
#         print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
# print("\nTraining Time (in minutes) =",(time()-time0)/60)

epoch: 0 training loss: 56.48072245717049 validation loss: 3.85497884452343 training accuracy: 0.8299583333333334 validation accuracy: 0.9510833333333333
epoch: 1 training loss: 12.291927568614483 validation loss: 2.842190206050873 training accuracy: 0.959625 validation accuracy: 0.9630833333333333
epoch: 2 training loss: 8.931119568645954 validation loss: 2.360059879720211 training accuracy: 0.9707708333333334 validation accuracy: 0.9693333333333334
epoch: 3 training loss: 6.553195860236883 validation loss: 2.008724622428417 training accuracy: 0.978375 validation accuracy: 0.9739166666666667
epoch: 4 training loss: 5.565003030002117 validation loss: 1.9605770744383335 training accuracy: 0.9822291666666667 validation accuracy: 0.9736666666666667
epoch: 5 training loss: 5.418654039502144 validation loss: 2.51988335698843 training accuracy: 0.9816666666666667 validation accuracy: 0.9680833333333333
epoch: 6 training loss: 5.132919965311885 validation loss: 1.9558661617338657 training acc

In [50]:
# epochs = 10
# for epoch in range(epochs):
#     model.eval()
#     val_loss = 0
#     correct = 0
    
#     with torch.no_grad():
#         for images, labels in valloader:
#             images, labels = images.to(device), labels.to(device)
#             # Flatten MNIST images into a 784 long vector
#             images = images.view(images.shape[0], -1)
#             #images = ball.projx(images.view(images.shape[0], -1))
#             output = model(images)
#             loss = criterion(output, labels)  
#             val_loss += loss.item()
#             correct += get_num_correct(output, labels)

#         tb.add_scalar("Validation Loss", val_loss, epoch)
#         tb.add_scalar("Validation Accuracy", correct/len(val_data), epoch)
#         print("epoch:", epoch, "total correct:", correct, "validation loss:",val_loss)
#         print("Validation Accuracy", correct/len(val_data))
# tb.close()

In [51]:
# images, labels = next(iter(testloader))
# #images, labels = images.to(device), labels.to(device)

# img = images[0].view(1, 784)
# #img = ball.projx(images[0].view(1, 784))
# #img_gpu = img.to(device)
# with torch.no_grad():
#     out = model(img)

# ps = out.cpu()
# print(ps)
# probab = list(ps.numpy()[0])
# print(probab)
# print("Predicted Digit =", probab.index(max(probab)))
# helper.view_classify(img.view(1, 28, 28), ps)

#Model Prediction and Model Accuracy

In [52]:
correct_count, all_count = 0, 0
for images,labels in testloader:
  images, labels = images.to(device), labels.to(device)
  for i in range(len(labels)):
    img = images[i].view(1, 784)
    #img = ball.projx(images[i].view(1, 784))
    with torch.no_grad():
        out = model(img)

    ps = out.cpu()
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.cpu().numpy()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1
    
print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

Number Of Images Tested = 10000

Model Accuracy = 0.9716


Current status of experiments:
1. Using just Hyperboic Linear modules, and with the appropriate self-tuned hyperparameters, and a batch size of 512, the average accuracy was around 90 percent
2. With the use of activation functions(ReLu, ReLu, then LogSoftMax at the output layer), (by applying the functions in the tangent space, then mapping it back to the hyperbolic space), we see an increase in the model accuracy to about 97-98 percent.
3. To account for the correct class probabilities , linear layer was used as the output layer instead, together with the crossentropy loss function.